In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
import time
import re

from selenium import webdriver

import warnings
warnings.filterwarnings('ignore')

In [2]:
driver = webdriver.Chrome()

In [3]:
driver.get('https://www.makemytrip.com/hotels/hotel-listing/?checkin=10142024&checkout=10162024&city=CTGOI&country=IN&locusId=CTGOI&locusType=city&regionNearByExp=3&roomStayQualifier=2e0e&rsc=1e2eundefinede&searchText=Goa')

In [190]:
soup = BeautifulSoup(driver.page_source)

In [191]:
# # Get scroll height
# import time
# last_height = driver.execute_script("return document.body.scrollHeight")

# while True:
#     # Scroll down to bottom
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

#     # Wait to load page
#     time.sleep(6)

#     # Calculate new scroll height and compare with last scroll height
#     new_height = driver.execute_script("return document.body.scrollHeight")
#     if new_height == last_height:
#         break
#     last_height = new_height

In [192]:
import time

# Get initial scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

for _ in range(500):
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(6)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        pass
    last_height = new_height


In [193]:
html = driver.page_source


In [194]:
soup = BeautifulSoup(html)

In [195]:
Names = []

for i in soup.find_all('div', class_='flexOne appendLeft20'):
    if re.findall("^[A-Za-z0-9 &',\-.a]+",i.text):
        Names.append(re.findall("^[A-Za-z0-9 &',\-.a]+",i.text)[0])
    else:
        Names.append(np.nan)

In [196]:
# for i in soup.find_all('div', class_='flexOne appendLeft20'):
#     print(i.text)

In [197]:
# for i in Names:
#     print(re.sub('MMT\sValueStaysTop', "",i))

In [198]:
Hotel = []
Location = []
Ratings = []
walk = []


for i in soup.find_all('div', class_='flexOne appendLeft20'):
    r = i.find('span', class_='wordBreak appendRight10')
    Hotel.append(r.text)
    
    r1 = i.find('span', class_='blueText')
    Location.append(r1.text)
    
    r2 = i.find('span', class_="ratingValue")
    Ratings.append(r2)
    
    near = i.find('span', class_="latoRegular")
    walk.append(near)
    
    
    
    
    """
    s = i.find('div', class_='pc__html')
    #print(s.text)
    print(re.findall("(\w+)\s\|",s.text))"""

In [199]:
Ratings = []
No_of_Ratings = []
No_of_Reviews = []


for i in soup.find_all('div',class_="textRight priceDetailsNewPers padding16"):
    if re.findall('(?:Very Good|Excellent|Good|Fair|Poor)(\d\.\d)',i.text):
        Ratings.append(re.findall('(?:Very Good|Excellent|Good|Fair|Poor)(\d\.\d)',i.text)[0])
    else:
        Ratings.append(np.nan)
    if re.findall('(\d+)\sRatings',i.text):
        No_of_Ratings.append(re.findall('(\d+)\sRatings',i.text)[0])
    else:
        No_of_Ratings.append(np.nan)   
    if re.findall('(\d+)\sreviews',i.text):
        No_of_Reviews.append(re.findall('(\d+)\sreviews',i.text)[0])
    else:
        No_of_Reviews.append(np.nan)     
        

In [200]:
Original_price = []
Discount_price = []
Taxes = []

for i in  soup.find_all('div',class_="textRight priceDetailsNewPers padding16"):
    if re.findall("₹(\s\d.{4})",i.text):
        Original_price.append(re.findall("₹(\s\d.{4})",i.text)[0])
    else:
        Original_price.append(np.nan)
    if re.findall("₹\s\d.{4}₹(\s\d.{4})",i.text):
        Discount_price.append(re.findall("₹\s\d.{4}₹(\s\d.{4})",i.text)[0])
    else:
        Discount_price.append(np.nan)  
    if re.findall("₹\s(\d\,?\d+)\staxes",i.text):
        Taxes.append(re.findall("₹\s(\d\,?\d+)\staxes",i.text)[0])
    else:
        Taxes.append(np.nan)

In [2]:
# nearest = []


# for i in soup.find_all('div', class_="makeFlex flexOne padding16 relative lftCol"):
#     #print(i.text)
#     near = i.find("span",class_="latoRegular")
#     if near:
#         nearest.append(near.get_text(strip=True))
#     else:
#         nearest.append(np.nan)
    

In [202]:
df_1 = pd.DataFrame({ "Hotel":Hotel,
               "Location":Location,
              "Ratings":Ratings,
              "No_of_Ratings":No_of_Ratings,
              "No_of_Reviews":No_of_Reviews,
              "Original_price":Original_price,
              "Discount_price":Discount_price,
              "Taxes":Taxes
              
})

In [203]:
df_1

,Hotel,Location,Ratings,No_of_Ratings,No_of_Reviews,Original_price,Discount_price,Taxes
0,Hotel Campal,Panjim,3.8,1164,646,"2,549","2,042",528
1,ibis Styles Goa Calangute - An Accor Brand,Calangute,4.1,4348,2615,"5,750",NaN,690
2,SinQ Prive,Candolim,4.0,2251,1239,"3,690","3,010",720
3,"Ginger Goa, Panjim",Panjim,3.9,5840,3566,"5,149",NaN,618
4,Calangute Heights by Stay ALYF | Luxury 1 BHK ...,Calangute,4.6,235,111,"6,500","2,657",736
...,...,...,...,...,...,...,...,...
874,"Mandrem Beach Resort, a member of Radisson Ind...",Mandrem Beach,4.8,54,23,"16,00",NaN,"2,880"
875,Marcello Villa | Rooms & Caretaker,Baga,NaN,314,166,999₹,201 t,201
876,Subhash Guest House,Baga,3.4,180,118,"1,000",911+,210
877,Villa Calangute Phase 12,Calangute,4.4,91,NaN,"30,60",NaN,"5,508"


In [220]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 879 entries, 0 to 878
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Hotel           879 non-null    object
 1   Location        879 non-null    object
 2   Ratings         848 non-null    object
 3   No_of_Ratings   863 non-null    object
 4   No_of_Reviews   713 non-null    object
 5   Original_price  879 non-null    object
 6   Discount_price  742 non-null    object
 7   Taxes           876 non-null    object
dtypes: object(8)
memory usage: 55.1+ KB


In [222]:
df_1.isnull().sum()

Hotel               0
Location            0
Ratings            31
No_of_Ratings      16
No_of_Reviews     166
Original_price      0
Discount_price    137
Taxes               3
dtype: int64

In [224]:
df_1.tail()

,Hotel,Location,Ratings,No_of_Ratings,No_of_Reviews,Original_price,Discount_price,Taxes
874,"Mandrem Beach Resort, a member of Radisson Ind...",Mandrem Beach,4.8,54,23,"16,00",NaN,"2,880"
875,Marcello Villa | Rooms & Caretaker,Baga,NaN,314,166,999₹,201 t,201
876,Subhash Guest House,Baga,3.4,180,118,"1,000",911+,210
877,Villa Calangute Phase 12,Calangute,4.4,91,NaN,"30,60",NaN,"5,508"
878,Baga Fantacia Beach Inn,Baga,3.4,247,156,"5,500","3,961",968


In [226]:
df_1.to_excel(r'C:\Users\DELL\Final Project\files\makemytripdf_1.xlsx')

## Run from here:

In [6]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [8]:
df_1 = pd.read_excel(r'C:\Users\DELL\Final Project\files\makemytripdf_1.xlsx')

In [9]:
df_1.drop(columns = 'Unnamed: 0', inplace = True)

In [10]:
df_1.head()

,Hotel,Location,Ratings,No_of_Ratings,No_of_Reviews,Original_price,Discount_price,Taxes
0,Hotel Campal,Panjim,3.8,1164.0,646.0,"2,549","2,042",528
1,ibis Styles Goa Calangute - An Accor Brand,Calangute,4.1,4348.0,2615.0,"5,750",NaN,690
2,SinQ Prive,Candolim,4.0,2251.0,1239.0,"3,690","3,010",720
3,"Ginger Goa, Panjim",Panjim,3.9,5840.0,3566.0,"5,149",NaN,618
4,Calangute Heights by Stay ALYF | Luxury 1 BHK ...,Calangute,4.6,235.0,111.0,"6,500","2,657",736


In [6]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 879 entries, 0 to 878
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Hotel           879 non-null    object 
 1   Location        879 non-null    object 
 2   Ratings         848 non-null    float64
 3   No_of_Ratings   863 non-null    float64
 4   No_of_Reviews   713 non-null    float64
 5   Original_price  879 non-null    object 
 6   Discount_price  742 non-null    object 
 7   Taxes           876 non-null    object 
dtypes: float64(3), object(5)
memory usage: 55.1+ KB


## Merging Hotel Types:

### Merging Hotel Type -1

In [19]:
df_2 = pd.read_excel(r'C:\Users\DELL\Final Project/files/buildingtype_1df.xlsx')

In [21]:
df_2.drop(columns = 'Unnamed: 0', inplace = True)

In [23]:
df = pd.merge(df_1, df_2, on = 'Hotel', how = 'left')
df

,Hotel,Location,Ratings,No_of_Ratings,No_of_Reviews,Original_price,Discount_price,Taxes,Building_Type
0,Hotel Campal,Panjim,3.8,1164.0,646.0,"2,549","2,042",528,Apartment
1,ibis Styles Goa Calangute - An Accor Brand,Calangute,4.1,4348.0,2615.0,"5,750",NaN,690,Apartment
2,SinQ Prive,Candolim,4.0,2251.0,1239.0,"3,690","3,010",720,Apartment
3,"Ginger Goa, Panjim",Panjim,3.9,5840.0,3566.0,"5,149",NaN,618,Apartment
4,Calangute Heights by Stay ALYF | Luxury 1 BHK ...,Calangute,4.6,235.0,111.0,"6,500","2,657",736,NaN
...,...,...,...,...,...,...,...,...,...
874,"Mandrem Beach Resort, a member of Radisson Ind...",Mandrem Beach,4.8,54.0,23.0,"16,00",NaN,"2,880",NaN
875,Marcello Villa | Rooms & Caretaker,Baga,NaN,314.0,166.0,999₹,201 t,201,NaN
876,Subhash Guest House,Baga,3.4,180.0,118.0,"1,000",911+,210,NaN
877,Villa Calangute Phase 12,Calangute,4.4,91.0,NaN,"30,60",NaN,"5,508",NaN


### Merging Hotel Type - 2

In [26]:
df_3 = pd.read_excel(r'C:\Users\DELL\Final Project/files/buildingtype_2df.xlsx')

In [28]:
df_3.drop(columns = 'Unnamed: 0', inplace = True)

In [30]:
df = pd.merge(df,df_3, on = 'Hotel', how = 'left')

df['Building_Type'] = df['Building_Type_x'].combine_first(df['Building_Type_y'])
df.drop(columns = ['Building_Type_x', 'Building_Type_y'], inplace = True)
df.tail(4)

,Hotel,Location,Ratings,No_of_Ratings,No_of_Reviews,Original_price,Discount_price,Taxes,Building_Type
875,Marcello Villa | Rooms & Caretaker,Baga,NaN,314.0,166.0,999₹,201 t,201,Villa
876,Subhash Guest House,Baga,3.4,180.0,118.0,"1,000",911+,210,NaN
877,Villa Calangute Phase 12,Calangute,4.4,91.0,NaN,"30,60",NaN,"5,508",Villa
878,Baga Fantacia Beach Inn,Baga,3.4,247.0,156.0,"5,500","3,961",968,Apartment


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 879 entries, 0 to 878
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Hotel           879 non-null    object 
 1   Location        879 non-null    object 
 2   Ratings         848 non-null    float64
 3   No_of_Ratings   863 non-null    float64
 4   No_of_Reviews   713 non-null    float64
 5   Original_price  879 non-null    object 
 6   Discount_price  742 non-null    object 
 7   Taxes           876 non-null    object 
 8   Building_Type   266 non-null    object 
dtypes: float64(3), object(6)
memory usage: 61.9+ KB


### Merging Hotel Type - 3

In [36]:
df_4 = pd.read_excel(r'C:\Users\DELL\Final Project/files/buildingtype_3df.xlsx')


In [38]:
df_4.drop(columns = 'Unnamed: 0', inplace = True)

In [40]:
df = pd.merge(df, df_4, on = 'Hotel', how = 'left')

df['Building_Type'] = df['Building_Type_x'].combine_first(df['Building_Type_y'])
df.drop(columns = ['Building_Type_x', 'Building_Type_y'], inplace = True)
df.tail(4)

,Hotel,Location,Ratings,No_of_Ratings,No_of_Reviews,Original_price,Discount_price,Taxes,Building_Type
875,Marcello Villa | Rooms & Caretaker,Baga,NaN,314.0,166.0,999₹,201 t,201,Villa
876,Subhash Guest House,Baga,3.4,180.0,118.0,"1,000",911+,210,NaN
877,Villa Calangute Phase 12,Calangute,4.4,91.0,NaN,"30,60",NaN,"5,508",Villa
878,Baga Fantacia Beach Inn,Baga,3.4,247.0,156.0,"5,500","3,961",968,Apartment


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 879 entries, 0 to 878
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Hotel           879 non-null    object 
 1   Location        879 non-null    object 
 2   Ratings         848 non-null    float64
 3   No_of_Ratings   863 non-null    float64
 4   No_of_Reviews   713 non-null    float64
 5   Original_price  879 non-null    object 
 6   Discount_price  742 non-null    object 
 7   Taxes           876 non-null    object 
 8   Building_Type   458 non-null    object 
dtypes: float64(3), object(6)
memory usage: 61.9+ KB


In [44]:
df.Building_Type.isnull().sum()

421

In [46]:
df.Building_Type.value_counts()

Building_Type
Apartment    240
Resort       192
Villa         26
Name: count, dtype: int64

### Merging Hotel Type - 4

In [49]:
df_5 = pd.read_excel(r'C:\Users\DELL\Final Project/files/buildingtype_4df.xlsx')


In [51]:
df_5.drop(columns = 'Unnamed: 0', inplace = True)

In [53]:
df = pd.merge(df, df_5, on = 'Hotel', how = 'left')

df['Building_Type'] = df['Building_Type_x'].combine_first(df['Building_Type_y'])
df.drop(columns = ['Building_Type_x', 'Building_Type_y'], inplace = True)
df.tail(4)

,Hotel,Location,Ratings,No_of_Ratings,No_of_Reviews,Original_price,Discount_price,Taxes,Building_Type
875,Marcello Villa | Rooms & Caretaker,Baga,NaN,314.0,166.0,999₹,201 t,201,Villa
876,Subhash Guest House,Baga,3.4,180.0,118.0,"1,000",911+,210,NaN
877,Villa Calangute Phase 12,Calangute,4.4,91.0,NaN,"30,60",NaN,"5,508",Villa
878,Baga Fantacia Beach Inn,Baga,3.4,247.0,156.0,"5,500","3,961",968,Apartment


In [55]:
df.Building_Type.value_counts()


Building_Type
Apartment    240
Resort       192
Hotel         32
Villa         26
Name: count, dtype: int64

In [57]:
df.Building_Type.isnull().sum()

389

### Merging Hotel Type - 5

In [67]:
df_6 = pd.read_excel(r'C:\Users\DELL\Final Project/files/buildingtype_5df.xlsx')

In [69]:
df_6.drop(columns = 'Unnamed: 0', inplace = True)


In [71]:
df = pd.merge(df, df_6, on = 'Hotel', how = 'left')

df['Building_Type'] = df['Building_Type_x'].combine_first(df['Building_Type_y'])
df.drop(columns = ['Building_Type_x', 'Building_Type_y'], inplace = True)
df.tail(4)

,Hotel,Location,Ratings,No_of_Ratings,No_of_Reviews,Original_price,Discount_price,Taxes,Building_Type
875,Marcello Villa | Rooms & Caretaker,Baga,NaN,314.0,166.0,999₹,201 t,201,Villa
876,Subhash Guest House,Baga,3.4,180.0,118.0,"1,000",911+,210,NaN
877,Villa Calangute Phase 12,Calangute,4.4,91.0,NaN,"30,60",NaN,"5,508",Villa
878,Baga Fantacia Beach Inn,Baga,3.4,247.0,156.0,"5,500","3,961",968,Apartment


In [73]:
df.Building_Type.value_counts()


Building_Type
Apartment    240
Resort       192
Homestay      60
Hotel         32
Villa         26
Name: count, dtype: int64

In [75]:
df.Building_Type.isnull().sum()


329

### Merging Hotel Type - 6


In [81]:
df_7= pd.read_excel(r'C:\Users\DELL\Final Project/files/buildingtype_6df.xlsx')


In [83]:
df_7.drop(columns = 'Unnamed: 0', inplace = True)


In [85]:
df = pd.merge(df, df_7, on = 'Hotel', how = 'left')

df['Building_Type'] = df['Building_Type_x'].combine_first(df['Building_Type_y'])
df.drop(columns = ['Building_Type_x', 'Building_Type_y'], inplace = True)
df.tail(4)

,Hotel,Location,Ratings,No_of_Ratings,No_of_Reviews,Original_price,Discount_price,Taxes,Building_Type
875,Marcello Villa | Rooms & Caretaker,Baga,NaN,314.0,166.0,999₹,201 t,201,Villa
876,Subhash Guest House,Baga,3.4,180.0,118.0,"1,000",911+,210,Guest House
877,Villa Calangute Phase 12,Calangute,4.4,91.0,NaN,"30,60",NaN,"5,508",Villa
878,Baga Fantacia Beach Inn,Baga,3.4,247.0,156.0,"5,500","3,961",968,Apartment


In [87]:
df.Building_Type.value_counts()


Building_Type
Apartment      240
Resort         192
Homestay        60
Hotel           32
Villa           26
Guest House     21
Name: count, dtype: int64

In [89]:
df.Building_Type.isnull().sum()

308

## Merging House Rule - 1

In [94]:
df_8 = pd.read_excel(r'C:\Users\DELL\Final Project/files/houserule_1df.xlsx') 

In [96]:
df_8.drop(columns = 'Unnamed: 0', inplace = True)

In [98]:
df = pd.merge(df, df_8, on = 'Hotel', how = 'left')

df.tail(4)

,Hotel,Location,Ratings,No_of_Ratings,No_of_Reviews,Original_price,Discount_price,Taxes,Building_Type,House_Rule
875,Marcello Villa | Rooms & Caretaker,Baga,NaN,314.0,166.0,999₹,201 t,201,Villa,Alcohol Allowed
876,Subhash Guest House,Baga,3.4,180.0,118.0,"1,000",911+,210,Guest House,Alcohol Allowed
877,Villa Calangute Phase 12,Calangute,4.4,91.0,NaN,"30,60",NaN,"5,508",Villa,Alcohol Allowed
878,Baga Fantacia Beach Inn,Baga,3.4,247.0,156.0,"5,500","3,961",968,Apartment,NaN


In [100]:
df.House_Rule.value_counts()


House_Rule
Alcohol Allowed    537
Name: count, dtype: int64

## Merging House Rule - 2

In [103]:
df_9 = pd.read_excel(r'C:\Users\DELL\Final Project/files/houserule_2df.xlsx')

In [105]:
df_9

,Unnamed: 0,Hotel,House_Rule
0,0,TreeHouse Neptune,Unmarried Couples Allowed
1,1,Elements by Rosetta,Unmarried Couples Allowed
2,2,"Ginger Goa, Panjim",Unmarried Couples Allowed
3,3,Hotel Rajdhani,Unmarried Couples Allowed
4,4,Hotel Campal,Unmarried Couples Allowed
...,...,...,...
1122,1122,Shunya Kaju Varo,Unmarried Couples Allowed
1123,1123,Candolim Alcove | Entire 1 BHK Apartment,Unmarried Couples Allowed
1124,1124,StayVista at Casa Ferrazi,Unmarried Couples Allowed
1125,1125,StayVista at Tarika's Olive Villa,Unmarried Couples Allowed


In [107]:
df_9.drop(columns = 'Unnamed: 0', inplace = True)


In [109]:
df = pd.merge(df, df_9, on = 'Hotel', how = 'left')

df['House_Rule'] = df['House_Rule_x'].combine_first(df['House_Rule_y'])
df.drop(columns = ['House_Rule_x', 'House_Rule_y'], inplace = True)

df.tail(4)

,Hotel,Location,Ratings,No_of_Ratings,No_of_Reviews,Original_price,Discount_price,Taxes,Building_Type,House_Rule
875,Marcello Villa | Rooms & Caretaker,Baga,NaN,314.0,166.0,999₹,201 t,201,Villa,Alcohol Allowed
876,Subhash Guest House,Baga,3.4,180.0,118.0,"1,000",911+,210,Guest House,Alcohol Allowed
877,Villa Calangute Phase 12,Calangute,4.4,91.0,NaN,"30,60",NaN,"5,508",Villa,Alcohol Allowed
878,Baga Fantacia Beach Inn,Baga,3.4,247.0,156.0,"5,500","3,961",968,Apartment,NaN


In [111]:
df[df['Hotel']=='Elements by Rosetta']

,Hotel,Location,Ratings,No_of_Ratings,No_of_Reviews,Original_price,Discount_price,Taxes,Building_Type,House_Rule
360,Elements by Rosetta,South Goa,4.8,237.0,120.0,"9,000","7,347","2,309",Resort,Alcohol Allowed


In [113]:
df.House_Rule.value_counts()

House_Rule
Alcohol Allowed              537
Unmarried Couples Allowed    159
Name: count, dtype: int64

## Merging House Rule - 3

In [124]:
df_10 = pd.read_excel(r'C:\Users\DELL\Final Project/files/houserule_3df.xlsx')


In [126]:
df_10.drop(columns = 'Unnamed: 0', inplace = True)


In [128]:
df = pd.merge(df, df_10, on = 'Hotel', how = 'left')

df['House_Rule'] = df['House_Rule_x'].combine_first(df['House_Rule_y'])
df.drop(columns = ['House_Rule_x', 'House_Rule_y'], inplace = True)

df.tail(4)

,Hotel,Location,Ratings,No_of_Ratings,No_of_Reviews,Original_price,Discount_price,Taxes,Building_Type,House_Rule
875,Marcello Villa | Rooms & Caretaker,Baga,NaN,314.0,166.0,999₹,201 t,201,Villa,Alcohol Allowed
876,Subhash Guest House,Baga,3.4,180.0,118.0,"1,000",911+,210,Guest House,Alcohol Allowed
877,Villa Calangute Phase 12,Calangute,4.4,91.0,NaN,"30,60",NaN,"5,508",Villa,Alcohol Allowed
878,Baga Fantacia Beach Inn,Baga,3.4,247.0,156.0,"5,500","3,961",968,Apartment,NaN


In [130]:
df.House_Rule.value_counts()

House_Rule
Alcohol Allowed              537
Unmarried Couples Allowed    159
Smoking Allowed               10
Name: count, dtype: int64

## Merging House Rule - 4

In [157]:
df_11 = pd.read_excel(r'C:\Users\DELL\Final Project/files/houserule_4df.xlsx')

In [159]:
df_11.drop(columns = 'Unnamed: 0', inplace = True)


In [161]:
# df.drop(columns = 'House_Rule_y', inplace = True)

In [163]:
# df.rename(columns = {'House_Rule_x': 'House_Rule'}, inplace = True)

In [165]:
# df

In [167]:
df = pd.merge(df, df_11, on = 'Hotel', how = 'left')

df['House_Rule'] = df['House_Rule_x'].combine_first(df['House_Rule_y'])
df.drop(columns = ['House_Rule_x', 'House_Rule_y'], inplace = True)

df.tail(4)

,Hotel,Location,Ratings,No_of_Ratings,No_of_Reviews,Original_price,Discount_price,Taxes,Building_Type,House_Rule
875,Marcello Villa | Rooms & Caretaker,Baga,NaN,314.0,166.0,999₹,201 t,201,Villa,Alcohol Allowed
876,Subhash Guest House,Baga,3.4,180.0,118.0,"1,000",911+,210,Guest House,Alcohol Allowed
877,Villa Calangute Phase 12,Calangute,4.4,91.0,NaN,"30,60",NaN,"5,508",Villa,Alcohol Allowed
878,Baga Fantacia Beach Inn,Baga,3.4,247.0,156.0,"5,500","3,961",968,Apartment,NaN


In [169]:
df.House_Rule.value_counts()

House_Rule
Alcohol Allowed              537
Unmarried Couples Allowed    159
Pets Allowed                  25
Smoking Allowed               10
Name: count, dtype: int64

In [171]:
df_11

,Hotel,House_Rule
0,Goa Marriott Resort & Spa,Pets Allowed
1,TreeHouse Blue Hotel & Serviced Apartments,Pets Allowed
2,SaffronStays Love Story,Pets Allowed
3,"Lazy Lagoon, A Lemon Tree Resort, Goa, Baga",Pets Allowed
4,Hungry House,Pets Allowed
...,...,...
450,CHICOS BLISS 2BHK CANDOLIM,Pets Allowed
451,Luxury Private Pool Serviced Villa,Pets Allowed
452,Hotel O Neelratna,Pets Allowed
453,Sk villa goa,Pets Allowed


In [182]:
df.to_excel(r'C:\Users\DELL\Final Project/files/final_df.xlsx')

df

In [174]:
df

,Hotel,Location,Ratings,No_of_Ratings,No_of_Reviews,Original_price,Discount_price,Taxes,Building_Type,House_Rule
0,Hotel Campal,Panjim,3.8,1164.0,646.0,"2,549","2,042",528,Apartment,Alcohol Allowed
1,ibis Styles Goa Calangute - An Accor Brand,Calangute,4.1,4348.0,2615.0,"5,750",NaN,690,Apartment,Unmarried Couples Allowed
2,SinQ Prive,Candolim,4.0,2251.0,1239.0,"3,690","3,010",720,Apartment,Alcohol Allowed
3,"Ginger Goa, Panjim",Panjim,3.9,5840.0,3566.0,"5,149",NaN,618,Apartment,Alcohol Allowed
4,Calangute Heights by Stay ALYF | Luxury 1 BHK ...,Calangute,4.6,235.0,111.0,"6,500","2,657",736,NaN,Unmarried Couples Allowed
...,...,...,...,...,...,...,...,...,...,...
874,"Mandrem Beach Resort, a member of Radisson Ind...",Mandrem Beach,4.8,54.0,23.0,"16,00",NaN,"2,880",NaN,NaN
875,Marcello Villa | Rooms & Caretaker,Baga,NaN,314.0,166.0,999₹,201 t,201,Villa,Alcohol Allowed
876,Subhash Guest House,Baga,3.4,180.0,118.0,"1,000",911+,210,Guest House,Alcohol Allowed
877,Villa Calangute Phase 12,Calangute,4.4,91.0,NaN,"30,60",NaN,"5,508",Villa,Alcohol Allowed


In [180]:
df.isnull().sum()

Hotel               0
Location            0
Ratings            31
No_of_Ratings      16
No_of_Reviews     166
Original_price      0
Discount_price    137
Taxes               3
Building_Type     308
House_Rule        148
dtype: int64

## Merging star category - 1:


In [39]:
df = pd.read_excel(r'C:\Users\DELL\Final Project/files/final_df.xlsx')

In [40]:
df.drop(columns = 'Unnamed: 0', inplace = True)
df.head()

,Hotel,Location,Ratings,No_of_Ratings,No_of_Reviews,Original_price,Discount_price,Taxes,Building_Type,House_Rule
0,Hotel Campal,Panjim,3.8,1164.0,646.0,"2,549","2,042",528,Apartment,Alcohol Allowed
1,ibis Styles Goa Calangute - An Accor Brand,Calangute,4.1,4348.0,2615.0,"5,750",NaN,690,Apartment,Unmarried Couples Allowed
2,SinQ Prive,Candolim,4.0,2251.0,1239.0,"3,690","3,010",720,Apartment,Alcohol Allowed
3,"Ginger Goa, Panjim",Panjim,3.9,5840.0,3566.0,"5,149",NaN,618,Apartment,Alcohol Allowed
4,Calangute Heights by Stay ALYF | Luxury 1 BHK ...,Calangute,4.6,235.0,111.0,"6,500","2,657",736,NaN,Unmarried Couples Allowed


In [43]:
df_12 = pd.read_excel(r'C:\Users\DELL\Final Project/files/star_category_1df.xlsx')

In [45]:
df_12.drop(columns = 'Unnamed: 0', inplace = True)
df_12

,Hotel,hotel_type
0,Hotel Rajdhani,3 star
1,SinQ Beach Resort,3 star
2,Calangute Heights by Stay ALYF | Luxury 1 BHK ...,3 star
3,Carnival De Goa - Luxury Apartment near Candolim,3 star
4,Laguna Anjuna - A Country House Resort in Anju...,3 star
...,...,...
843,Rainforest Enclave Private Terrace 2BHK -CF1,3 star
844,FabHotel Prime Fair stayz,3 star
845,Hotel O Om Babu Guest House,3 star
846,Hotel O Ferns Guest House,3 star


In [49]:
df = pd.merge(df, df_12, on = 'Hotel', how = 'left')

df['hotel_type'] = df['hotel_type_x'].combine_first(df['hotel_type_y'])
df.drop(columns = ['hotel_type_x', 'hotel_type_y'], inplace = True)

df.tail(4)

,Hotel,Location,Ratings,No_of_Ratings,No_of_Reviews,Original_price,Discount_price,Taxes,Building_Type,House_Rule,hotel_type
875,Marcello Villa | Rooms & Caretaker,Baga,NaN,314.0,166.0,999₹,201 t,201,Villa,Alcohol Allowed,NaN
876,Subhash Guest House,Baga,3.4,180.0,118.0,"1,000",911+,210,Guest House,Alcohol Allowed,NaN
877,Villa Calangute Phase 12,Calangute,4.4,91.0,NaN,"30,60",NaN,"5,508",Villa,Alcohol Allowed,NaN
878,Baga Fantacia Beach Inn,Baga,3.4,247.0,156.0,"5,500","3,961",968,Apartment,NaN,3 star


In [76]:
df.isnull().sum()

Hotel               0
Location            0
Ratings            31
No_of_Ratings      16
No_of_Reviews     166
Original_price      0
Discount_price    137
Taxes               3
Building_Type     308
House_Rule        148
hotel_type        327
dtype: int64

## Merging star category - 2:

In [54]:
df_13 = pd.read_excel(r'C:\Users\DELL\Final Project/files/star_category_2df.xlsx')


In [56]:
df_13.drop(columns = 'Unnamed: 0', inplace = True)

In [58]:
df_13.tail()

,Hotel,hotel_type
237,La Zamora - 3 BHK Private Pool Villa at Vagato...,4 star
238,CASA NERUL VILLA,4 star
239,Nyast Valley Resort,4 star
240,StayVista at White Whale,4 star
241,SWAPNAGANDHA RESORT,4 star


In [60]:
df = pd.merge(df, df_13, on = 'Hotel', how = 'left')

df['hotel_type'] = df['hotel_type_x'].combine_first(df['hotel_type_y'])
df.drop(columns = ['hotel_type_x', 'hotel_type_y'], inplace = True)


In [62]:

df.tail(4)

,Hotel,Location,Ratings,No_of_Ratings,No_of_Reviews,Original_price,Discount_price,Taxes,Building_Type,House_Rule,hotel_type
876,Marcello Villa | Rooms & Caretaker,Baga,NaN,314.0,166.0,999₹,201 t,201,Villa,Alcohol Allowed,NaN
877,Subhash Guest House,Baga,3.4,180.0,118.0,"1,000",911+,210,Guest House,Alcohol Allowed,NaN
878,Villa Calangute Phase 12,Calangute,4.4,91.0,NaN,"30,60",NaN,"5,508",Villa,Alcohol Allowed,NaN
879,Baga Fantacia Beach Inn,Baga,3.4,247.0,156.0,"5,500","3,961",968,Apartment,NaN,3 star


In [78]:
df.isnull().sum()

Hotel               0
Location            0
Ratings            31
No_of_Ratings      16
No_of_Reviews     166
Original_price      0
Discount_price    137
Taxes               3
Building_Type     308
House_Rule        148
hotel_type        327
dtype: int64


## Merging star category - 3:

In [66]:
df_14 = pd.read_excel(r'C:\Users\DELL\Final Project/files/star_category_3df.xlsx')


In [68]:
df_14.drop(columns = 'Unnamed: 0', inplace = True)


In [70]:
df_14.tail()


,Hotel,hotel_type
84,"ama Stays & Trails | Moira by the Church, Goa",5 star
85,Sere Resort Goa,5 star
86,Staymaster Sea La Vie Infinity Pool Sea View,5 star
87,Rainforest Casa Oriana,5 star
88,Rainforest Azaya,5 star


In [72]:
df = pd.merge(df, df_14, on = 'Hotel', how = 'left')

df['hotel_type'] = df['hotel_type_x'].combine_first(df['hotel_type_y'])
df.drop(columns = ['hotel_type_x', 'hotel_type_y'], inplace = True)


In [74]:
df.tail(4)


,Hotel,Location,Ratings,No_of_Ratings,No_of_Reviews,Original_price,Discount_price,Taxes,Building_Type,House_Rule,hotel_type
876,Marcello Villa | Rooms & Caretaker,Baga,NaN,314.0,166.0,999₹,201 t,201,Villa,Alcohol Allowed,NaN
877,Subhash Guest House,Baga,3.4,180.0,118.0,"1,000",911+,210,Guest House,Alcohol Allowed,NaN
878,Villa Calangute Phase 12,Calangute,4.4,91.0,NaN,"30,60",NaN,"5,508",Villa,Alcohol Allowed,NaN
879,Baga Fantacia Beach Inn,Baga,3.4,247.0,156.0,"5,500","3,961",968,Apartment,NaN,3 star


In [80]:
df.isnull().sum()

Hotel               0
Location            0
Ratings            31
No_of_Ratings      16
No_of_Reviews     166
Original_price      0
Discount_price    137
Taxes               3
Building_Type     308
House_Rule        148
hotel_type        327
dtype: int64

In [85]:
df.hotel_type.value_counts()

hotel_type
3 star    353
4 star    145
5 star     55
Name: count, dtype: int64

## Saving final df to excel

In [90]:
df.to_excel(r'C:\Users\DELL\Final Project/files/final_df.xlsx')

## Reading final df

In [4]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [6]:
df = pd.read_excel(r'C:\Users\DELL\Final Project/files/final_df.xlsx')

In [8]:
df.head()

,Unnamed: 0,Hotel,Location,Ratings,No_of_Ratings,No_of_Reviews,Original_price,Discount_price,Taxes,Building_Type,House_Rule,hotel_type
0,0,Hotel Campal,Panjim,3.8,1164.0,646.0,"2,549","2,042",528,Apartment,Alcohol Allowed,NaN
1,1,ibis Styles Goa Calangute - An Accor Brand,Calangute,4.1,4348.0,2615.0,"5,750",NaN,690,Apartment,Unmarried Couples Allowed,4 star
2,2,SinQ Prive,Candolim,4.0,2251.0,1239.0,"3,690","3,010",720,Apartment,Alcohol Allowed,5 star
3,3,"Ginger Goa, Panjim",Panjim,3.9,5840.0,3566.0,"5,149",NaN,618,Apartment,Alcohol Allowed,3 star
4,4,Calangute Heights by Stay ALYF | Luxury 1 BHK ...,Calangute,4.6,235.0,111.0,"6,500","2,657",736,NaN,Unmarried Couples Allowed,3 star
